# Comparison of the different implemented methods

## Libraries importation

In [ ]:
# Please use these lines in every notebook you create

import os
import sys

# Get the current working directory
current_dir = os.getcwd()

# Get the parent directory of `maps` (which is `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Add `src` to the module search path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from global_variables import *
from python_scripts.miscellaneaous.data_processing import extract_data

In [ ]:
import pandas as pd
import folium.features
import numpy as np
from networkx import Graph
from itertools import permutations
from tqdm import tqdm

from python_scripts.graphs.graphs_creation import delaunay_graph, gabriel_graph, kNN_graph
from python_scripts.graphs.real_neighbors import find_real_neighbors
from python_scripts.neighbours_criteria.simple_criteria import distance_criterion, angle_criterion, quadrant_criterion
from python_scripts.neighbours_criteria.enhanced_criteria import distance_criterion_enhanced, angle_criterion_enhanced, quadrant_criterion_enhanced, quadrant_criterion_enhanced_v2
from python_scripts.city.city_utils import mean_distance_to_NN
from python_scripts.ihm.maps.map_neighbours import add_graph_edges, addLegend
from python_scripts.city.city_utils import mean_distance_choice

## Database import and data extraction

In [ ]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=",")
df = extract_data(df, provider=PROVIDER, techno=TECHNO, region=REGION, min_info=True)
df.head()

In [ ]:
df_azimuth_raw = pd.read_csv("../../database/data_azimuth_freq.csv", sep=";", decimal=",", dtype={'id_station_anfr':str})

dep_codes = ['027', '076', '014', '050', '061']
df_azimuth = pd.DataFrame(columns=df_azimuth_raw.columns)
for row in df_azimuth_raw.values:
    if(row[0][0:3] in dep_codes):
        df_azimuth = pd.concat([df_azimuth,pd.DataFrame([row],columns=df_azimuth_raw.columns)],ignore_index=True)
        
df_azimuth.head()

## Graphs creation

In [ ]:
del_G, pos = delaunay_graph(df)
print(len(del_G.edges))

mean_distances = mean_distance_to_NN(df[['x', 'y']], n_neighbours=N_NEIGH)

from python_scripts.neighbours_criteria.miscellaneous_for_neighbouring import compute_angles
from copy import deepcopy
ref_point = '0762290045'
adj = [edge[1] for edge in deepcopy(del_G).edges(ref_point)]

azimuths = np.unique(df_azimuth.loc[df_azimuth['id_station_anfr']==ref_point,'angle_azimuth'])

angles = compute_angles(ref_point, adj, pos)

quadrants = dict()
nb_az = len(azimuths)

if(nb_az>0):
    for id, az in enumerate(azimuths):
        next_id = (id+1) % nb_az
        next_az = azimuths[next_id]
        print(az, next_az)
        biss = int(((((next_az - az) / 2) % 180) + az) % 360)
        print(az, biss, next_az)
        quadrants[f"{biss}_{next_az}"] = [adj[k] for k in np.where((angles >= biss) & (angles < next_az))[0]]
        quadrants[f"{az}_{biss}"] = [adj[k] for k in np.where((angles >= az) & (angles < biss))[0]]
if(nb_az > 4):
    print(angles)
    print(nb_az, ref_point, azimuths)
    print(quadrants, '\n')

delG_enhC_v2 = quadrant_criterion_enhanced_v2(del_G, pos, MEAN_DISTANCE_PARAMS, mean_distances, df_azimuth=df_azimuth)

In [ ]:
gab_G, _ = gabriel_graph(df)
print(len(gab_G.edges))

In [ ]:
K_1=7
kNN_G_1, _ = kNN_graph(df, k=K_1)
print(len(kNN_G_1.edges))

In [ ]:
K_2=15
kNN_G_2, _ = kNN_graph(df, k=K_2)
print(len(kNN_G_2.edges))

## City detection

In [ ]:
mean_distances = mean_distance_to_NN(df[['x', 'y']], n_neighbours=N_NEIGH)

## Application of the criteria

def permutations_simple_criteria(base_graph: Graph, angle_criterion, distance_criterion, quadrant_criterion):
    res = []
    for perm in permutations([angle_criterion, distance_criterion, quadrant_criterion]):
        neigh_G = perm[0](base_graph, pos, max_distance=MAX_DISTANCE, min_angle=MIN_ANGLE)
        neigh_G = perm[1](neigh_G, pos, max_distance=MAX_DISTANCE, min_angle=MIN_ANGLE)
        neigh_G = perm[2](neigh_G, pos, max_distance=MAX_DISTANCE, min_angle=MIN_ANGLE)
        print(str(len(neigh_G.edges)) + '\n')
        res.append(neigh_G)

    return res

def permutations_enhanced_criteria(base_graph: Graph, angle_criterion, distance_criterion, quadrant_criterion, mean_distances):
    res = []
    for perm in permutations([angle_criterion, distance_criterion, quadrant_criterion]):
        neigh_G = perm[0](base_graph, pos, params=MEAN_DISTANCE_PARAMS, mean_distance_to_NN=mean_distances)
        neigh_G = perm[1](neigh_G, pos, params=MEAN_DISTANCE_PARAMS, mean_distance_to_NN=mean_distances)
        neigh_G = perm[2](neigh_G, pos, params=MEAN_DISTANCE_PARAMS, mean_distance_to_NN=mean_distances)
        print(str(len(neigh_G.edges)) + '\n')
        res.append(neigh_G)

    return res

In [ ]:
from types import NoneType


def permutations_criteria(base_graph: Graph, angle_criterion, distance_criterion, quadrant_criterion, param1, param2, **kwargs):
    df_az = kwargs.get('df_azimuth', None)

    criteria = {'a': angle_criterion, 'd': distance_criterion, 'q': quadrant_criterion}
    
    res = {}
    if(type(df_az)!=NoneType):
        for perm in tqdm(permutations(criteria, 3), desc="permutations - 3"):
            neigh_G = criteria[perm[0]](base_graph, pos, param1, param2, df_azimuth=df_az)
            neigh_G = criteria[perm[1]](neigh_G, pos, param1, param2, df_azimuth=df_az)
            neigh_G = criteria[perm[2]](neigh_G, pos, param1, param2, df_azimuth=df_az)
            print(f"{perm[0] + perm[1] + perm[2]}: {len(neigh_G.edges)}\n")
            res[f"{perm[0] + perm[1] + perm[2]}"] = neigh_G
        for perm in tqdm(permutations(criteria, 2), desc="permutations - 2"):
            neigh_G = criteria[perm[0]](base_graph, pos, param1, param2, df_azimuth=df_az)
            neigh_G = criteria[perm[1]](neigh_G, pos, param1, param2, df_azimuth=df_az)
            print(f"{perm[0] + perm[1]}: {len(neigh_G.edges)}\n")
            res[f"{perm[0] + perm[1]}"] = neigh_G
    else:
        for perm in tqdm(permutations(criteria, 3), desc="permutations - 3"):
            neigh_G = criteria[perm[0]](base_graph, pos, param1, param2)
            neigh_G = criteria[perm[1]](neigh_G, pos, param1, param2)
            neigh_G = criteria[perm[2]](neigh_G, pos, param1, param2)
            print(f"{perm[0] + perm[1] + perm[2]}: {len(neigh_G.edges)}\n")
            res[f"{perm[0] + perm[1] + perm[2]}"] = neigh_G
        for perm in tqdm(permutations(criteria, 2), desc="permutations - 2"):
            neigh_G = criteria[perm[0]](base_graph, pos, param1, param2)
            neigh_G = criteria[perm[1]](neigh_G, pos, param1, param2)
            print(f"{perm[0] + perm[1]}: {len(neigh_G.edges)}\n")
            res[f"{perm[0] + perm[1]}"] = neigh_G

    return res

In [ ]:
delG_simC = permutations_criteria(del_G, angle_criterion, distance_criterion, quadrant_criterion, MAX_DISTANCE, MIN_ANGLE)

In [ ]:
delG_enhC = permutations_criteria(del_G, angle_criterion_enhanced, distance_criterion_enhanced, quadrant_criterion_enhanced, MEAN_DISTANCE_PARAMS, mean_distances)

In [ ]:
delG_enhC_v2 = permutations_criteria(kNN_G_2, angle_criterion_enhanced, distance_criterion_enhanced, quadrant_criterion_enhanced_v2, MEAN_DISTANCE_PARAMS, mean_distances, df_azimuth=df_azimuth)

In [ ]:
gabG_enhC = permutations_criteria(gab_G, angle_criterion_enhanced, distance_criterion_enhanced, quadrant_criterion_enhanced, MEAN_DISTANCE_PARAMS, mean_distances)

In [ ]:
kNNG_enhC_1 = permutations_criteria(kNN_G_1, angle_criterion_enhanced, distance_criterion_enhanced, quadrant_criterion_enhanced, MEAN_DISTANCE_PARAMS, mean_distances)

In [ ]:
kNNG_enhC_2 = permutations_criteria(kNN_G_2, angle_criterion_enhanced, distance_criterion_enhanced, quadrant_criterion_enhanced, MEAN_DISTANCE_PARAMS, mean_distances)

In [ ]:
nei_G, edge_info = find_real_neighbors(df, df_azimuth, del_G, pos)
nei_G = distance_criterion_enhanced(nei_G, pos, MEAN_DISTANCE_PARAMS, mean_distances)
nei_G = angle_criterion_enhanced(nei_G, pos, MEAN_DISTANCE_PARAMS, mean_distances)

## Comparison with real neigh

In [ ]:
def neigh_method_dist(G_1, G_2):
    alpha = 0
    beta = 0
    for edge in G_1.edges:
        if(not(edge in G_2.edges)):
            alpha += 1
    for edge in G_2.edges:
        if(not(edge in G_1.edges)):
            beta += 1
    return alpha, beta, (alpha+beta)

In [ ]:
alpha, beta, delta = neigh_method_dist(nei_G, del_G)
print(f'Delaunay: & $alpha = {alpha}, beta = {beta}, \Delta = {delta}$')
alpha, beta, delta = neigh_method_dist(nei_G, gab_G)
print(f'Gabriel graph: & $alpha = {alpha}, beta = {beta}, \Delta = {delta}$')
alpha, beta, delta = neigh_method_dist(nei_G, kNN_G_1)
print(f'{K_1}-NN: & $alpha = {alpha}, beta = {beta}, \Delta = {delta}$')
alpha, beta, delta = neigh_method_dist(nei_G, kNN_G_2)
print(f'{K_2}-NN: & $alpha = {alpha}, beta = {beta}, \Delta = {delta}$')

In [ ]:
for key in delG_simC:
    alpha, beta, delta = neigh_method_dist(nei_G, delG_simC[key])
    print(f'Delaunay and simple-criteria [{key}]: & $alpha = {alpha}, beta = {beta}, \Delta = {delta}$')
    alpha, beta, delta = neigh_method_dist(nei_G, delG_enhC[key])
    print(f'Delaunay and enhanced-criteria [{key}]: & $alpha = {alpha}, beta = {beta}, \Delta = {delta}$')
    alpha, beta, delta = neigh_method_dist(nei_G, delG_enhC_v2[key])
    print(f'Delaunay and enhanced-criteria\_v2 [{key}]: & $alpha = {alpha}, beta = {beta}, \Delta = {delta}$')
    alpha, beta, delta = neigh_method_dist(nei_G, gabG_enhC[key])
    print(f'Gabriel graph and enhanced-criteria [{key}]: & $alpha = {alpha}, beta = {beta}, \Delta = {delta}$')
    alpha, beta, delta = neigh_method_dist(nei_G, kNNG_enhC_1[key])
    print(f'{K_1}-NN and enhanced-criteria [{key}]: & $alpha = {alpha}, beta = {beta}, \Delta = {delta}$')
    alpha, beta, delta = neigh_method_dist(nei_G, kNNG_enhC_2[key])
    print(f'{K_2}-NN and enhanced-criteria [{key}]: & $alpha = {alpha}, beta = {beta}, \Delta = {delta}$\n')

## Map creation

In [ ]:
def create_FeatureGroup(map, graphs_dict: dict, name: str):
    fg = {}
    for graph_key in graphs_dict:
        fg[graph_key] = folium.FeatureGroup(f"Edges - {name} [{graph_key}] ({len(graphs_dict[graph_key].edges)})", show=False).add_to(map)

    return fg

In [ ]:
# Function to generate edge info popup with visualization
def generate_edge_info_popup_with_visualization(edge, df, edge_info):
    bs_id, neigh_id = edge
    bs_info = df.loc[bs_id]
    neigh_info = df.loc[neigh_id]

    azimuths_info = edge_info.get((bs_id, neigh_id), {"station": [None, None], "neighbor": [None, None]})

    popup_text = (
        f"<b>Base Station {bs_id} - {bs_info['nom_com']}</b><br>"
        f"Latitude: {bs_info['latitude']}, Longitude: {bs_info['longitude']}<br>"
        f"Coverage Azimuths: {azimuths_info['station'][1]}°<br>"
        f"<br><b>Neighbor Station {neigh_id} - {neigh_info['nom_com']}</b><br>"
        f"Latitude: {neigh_info['latitude']}, Longitude: {neigh_info['longitude']}<br>"
        f"Coverage Azimuths: {azimuths_info['neighbor'][1]}°<br>"
    )

    return popup_text

In [ ]:
# Function to add edges with interactive information
def add_graph_edges_with_visualization(G: Graph, df: pd.DataFrame, fg: folium.FeatureGroup, edge_info: dict):
    for edge in G.edges:
        stations = df.loc[[edge[0], edge[1]], ['latitude', 'longitude']].values
        popup_text = generate_edge_info_popup_with_visualization(edge, df, edge_info)
        popup = folium.Popup(popup_text, max_width=300)
        line = folium.PolyLine(stations, color="#AAA662", weight=2.5, opacity=1)
        line.add_child(popup)
        line.add_to(fg)

In [ ]:
map = folium.Map(location=list(np.mean(df[['latitude','longitude']], axis=0)), zoom_start=8.5, tiles="Cartodb Positron")

edges_del = folium.FeatureGroup(f"Edges - Delaunay triangulation ({len(del_G.edges)})", show=False).add_to(map)
edges_gab = folium.FeatureGroup(f"Edges - Gab graph ({len(gab_G.edges)})", show=False).add_to(map)
edges_kNN_1 = folium.FeatureGroup(f"Edges - {K_1}-NN graph ({len(kNN_G_1.edges)})", show=False).add_to(map)
edges_kNN_2 = folium.FeatureGroup(f"Edges - {K_2}-NN graph ({len(kNN_G_2.edges)})", show=False).add_to(map)
add_graph_edges(del_G, Graph(), df, edges_del, colour="blue")
add_graph_edges(gab_G, Graph(), df, edges_gab, colour="green")
add_graph_edges(kNN_G_1, Graph(), df, edges_kNN_1, colour="red")
add_graph_edges(kNN_G_2, Graph(), df, edges_kNN_2, colour="magenta")

edges_del_simC = create_FeatureGroup(map, delG_simC, "delaunay&simple-criteria")
edges_del_enhC = create_FeatureGroup(map, delG_enhC, "delaunay&enhanced-criteria")
edges_del_enhC_v2 = create_FeatureGroup(map, delG_enhC_v2, f"delaunay&enhanced-criteria_v2")
edges_gab_enhC = create_FeatureGroup(map, gabG_enhC, "gabriel-graph&enhanced-criteria")
edges_kNN_enhC_1 = create_FeatureGroup(map, kNNG_enhC_1, f"{K_1}-NN&enhanced-criteria")
edges_kNN_enhC_2 = create_FeatureGroup(map, kNNG_enhC_2, f"{K_2}-NN&enhanced-criteria")
col = {'adq': '#7e1e9c', 'aqd': '#95d0fc', 'daq': '#06c2ac', 'dqa': '#06470c', 'qad': '#ff796c', 'qda': '#dbb40c', 'ad': 'blue', 'aq': 'green', 'da': 'red', 'dq': 'magenta', 'qa': 'yellow', 'qd': 'pink'}
for crit_perm in delG_simC:
    add_graph_edges(delG_simC[crit_perm], Graph(), df, edges_del_simC[crit_perm], colour=col[crit_perm])
for crit_perm in delG_enhC_v2:
    add_graph_edges(delG_enhC_v2[crit_perm], Graph(), df, edges_del_enhC_v2[crit_perm], colour=col[crit_perm])
col = {'adq': '#15b01a', 'aqd': '#029386', 'daq': '#929591', 'dqa': '#c79fef', 'qad': '#ae7181', 'qda': '#e6daa6', 'ad': 'blue', 'aq': 'green', 'da': 'red', 'dq': 'magenta', 'qa': 'yellow', 'qd': 'pink'}
for crit_perm in delG_enhC:
    add_graph_edges(delG_enhC[crit_perm], Graph(), df, edges_del_enhC[crit_perm], colour=col[crit_perm])
col = {'adq': '#0504aa', 'aqd': '#89fe05', 'daq': '#c7fdb5', 'dqa': '#840000', 'qad': '#137e6d', 'qda': '#b9a281', 'ad': 'blue', 'aq': 'green', 'da': 'red', 'dq': 'magenta', 'qa': 'yellow', 'qd': 'pink'}
for crit_perm in gabG_enhC:
    add_graph_edges(gabG_enhC[crit_perm], Graph(), df, edges_gab_enhC[crit_perm], colour=col[crit_perm])
col = {'adq': '#be03fd', 'aqd': '#a0450e', 'daq': '#4e7496', 'dqa': '#04d9ff', 'qad': '#fbeeac', 'qda': '#7b0323', 'ad': 'blue', 'aq': 'green', 'da': 'red', 'dq': 'magenta', 'qa': 'yellow', 'qd': 'pink'}
for crit_perm in kNNG_enhC_1:
    add_graph_edges(kNNG_enhC_1[crit_perm], Graph(), df, edges_kNN_enhC_1[crit_perm], colour=col[crit_perm])
col = {'adq': '#a87900', 'aqd': '#fc2647', 'daq': '#bff128', 'dqa': '#ca7b80', 'qad': '#f4320c', 'qda': '#7a6a4f', 'ad': 'blue', 'aq': 'green', 'da': 'red', 'dq': 'magenta', 'qa': 'yellow', 'qd': 'pink'}
for crit_perm in kNNG_enhC_2:
    add_graph_edges(kNNG_enhC_2[crit_perm], Graph(), df, edges_kNN_enhC_2[crit_perm], colour=col[crit_perm])

edges_nei = folium.FeatureGroup(f"Edges - Neighbours graph [da] ({len(nei_G.edges)})", show=False).add_to(map)
add_graph_edges_with_visualization(nei_G, df, edges_nei, edge_info)

points = folium.FeatureGroup(f"Base stations ({len(df)})").add_to(map)

for bs_id, latitude, longitude in df[['latitude', 'longitude']].itertuples():
    popup_text = folium.Popup(
        f"Id_anfr: {bs_id}<br>"
        f"Commune: {df.loc[bs_id, 'nom_com']}<br>"
        f"3NN distance: {mean_distances.get(bs_id)}<br>"
        , max_width=150)

    color = mean_distance_choice(bs_id, mean_distances, MEAN_DISTANCE_PARAMS, 'colour')
    points.add_child(folium.CircleMarker(location=[latitude, longitude], color=color, radius=4, popup=popup_text, fillOpacity=1, fill=True))
folium.LayerControl().add_to(map)

addLegend(map, MEAN_DISTANCE_PARAMS)

map.save(f"../../out/maps/neighbours_finding/methods_comparison_v3.html")